# Tasks
This notebook contains the solutions for the Computational Theory tasks

# Task 1: Binary Representations

In this task, we implement four functions based on the FIPS 180-4 Secure Hash Standard:
- `rotl(x, n=1)`: Rotate bits of a 32-bit unsigned integer to the left by n positions.
- `rotr(x, n=1)`: Rotate bits of a 32-bit unsigned integer to the right by n positions.
- `ch(x, y, z)`: Choose bits from `y` when the corresponding bit in `x` is 1; otherwise, choose from `z`.
- `maj(x, y, z)`: Compute the majority vote for each bit position (i.e., output a 1 if at least two of the three inputs have a 1 in that bit).

**Reference:** FIPS 180-4, Secure Hash Standard (SHS)

In [1]:
# Define the function to rotate left (rotl) for a 32-bit unsigned integer.
def rotl(x, n=1):
    """
    Rotate the 32-bit unsigned integer x to the left by n bits.
    
    Parameters:
        x (int): 32-bit unsigned integer.
        n (int): Number of positions to rotate (default is 1).
    
    Returns:
        int: The result of rotating x to the left by n positions.
    """
    # Shift left by n, shift right by (32 - n), and combine with bitwise OR.
    # Use bitwise AND with 0xFFFFFFFF to ensure the result stays within 32 bits.
    return ((x << n) | (x >> (32 - n))) & 0xFFFFFFFF

# Define the function to rotate right (rotr) for a 32-bit unsigned integer.
def rotr(x, n=1):
    """
    Rotate the 32-bit unsigned integer x to the right by n bits.
    
    Parameters:
        x (int): 32-bit unsigned integer.
        n (int): Number of positions to rotate (default is 1).
    
    Returns:
        int: The result of rotating x to the right by n positions.
    """
    return ((x >> n) | (x << (32 - n))) & 0xFFFFFFFF

# Define the choice function (ch) that selects bits from y or z based on x.
def ch(x, y, z):
    """
    Choose bits from y or z based on the bits in x.
    
    For each bit position, if x has a 1, take the corresponding bit from y;
    otherwise, take the bit from z.
    
    Parameters:
        x, y, z (int): 32-bit unsigned integers.
    
    Returns:
        int: The result after selecting bits.
    """
    # (x AND y) gives bits from y where x is 1.
    # (~x AND z) gives bits from z where x is 0.
    return (x & y) ^ ((~x) & z)  # XOR here works like OR for non-overlapping bits

# Define the majority function (maj) which outputs the majority vote of bits.
def maj(x, y, z):
    """
    Compute the majority of the bits in x, y, and z.
    
    For each bit position, the output bit is 1 if at least two of the inputs
    have a 1 in that position.
    
    Parameters:
        x, y, z (int): 32-bit unsigned integers.
    
    Returns:
        int: The result of the majority vote.
    """
    # Using the formula: (x AND y) OR (x AND z) OR (y AND z)
    return (x & y) | (x & z) | (y & z)

## Testing the Functions

In the following cell, we will demonstrate example usages and simple tests for each of the functions:
- We'll provide sample inputs to check that our functions perform the expected bit rotations.
- We will test `ch` and `maj` to confirm they return the correct values based on binary logic.
- Each example is accompanied by comments to explain what is being tested.


In [2]:
# Test examples for the functions

# Example for rotl:
x = 0x12345678  # Example 32-bit number in hexadecimal
rotated_left = rotl(x, 4)
print("rotl(0x12345678, 4):", hex(rotated_left))
# Expected: The bits should be rotated 4 positions to the left

# Example for rotr:
rotated_right = rotr(x, 4)
print("rotr(0x12345678, 4):", hex(rotated_right))
# Expected: The bits should be rotated 4 positions to the right

# Example for ch:
# Let's choose arbitrary 32-bit numbers for testing.
a = 0b10101010101010101010101010101010  # Pattern: alternating bits
b = 0xFFFFFFFF  # All bits 1
c = 0x00000000  # All bits 0
chosen = ch(a, b, c)
print("ch(a, b, c):", bin(chosen))
# Expected: For each bit position, if bit in a is 1, bit from b (1) is chosen; else, bit from c (0)

# Example for maj:
# Using three arbitrary numbers.
x_val = 0b11001100110011001100110011001100
y_val = 0b10101010101010101010101010101010
z_val = 0b11110000111100001111000011110000
majority = maj(x_val, y_val, z_val)
print("maj(x, y, z):", bin(majority))
# Expected: For each bit position, the output is 1 if at least two of x, y, and z have a 1.


rotl(0x12345678, 4): 0x23456781


rotr(0x12345678, 4): 0x81234567
ch(a, b, c): 0b10101010101010101010101010101010
maj(x, y, z): 0b11101000111010001110100011101000


# Task 2: Hash Functions

In this task, we convert a hash function from C (from *The C Programming Language* by Kernighan and Ritchie) into Python. The original C code is:

```c
unsigned hash(char *s) {
    unsigned hashval;
    for (hashval = 0; *s != '\0'; s++)
        hashval = *s + 31 * hashval;
    return hashval % 101;
}


**Purpose:**  
Define the Python version of the hash function. Since `hash` is a built-in in Python, we use the name `custom_hash`.

**What to Do:**  
Immediately below the Markdown cell from Step 1, insert a new **Code cell** and paste the following code:
